# Enzyme function prediction using neural networks

In this homework, we will design several neural networks with different architectures for Enzyme function predictions from protein sequences.

Enzyme functions can be represented by Enzyme Commission (EC) numbers. In this problem, each enzyme (protein) in the training or test set is labeled with exactly one EC number. There are a total of 200 distinct EC numbers appeared in the dataset. So this task can be formulated as a single-label multi-class classification problem.

To begin with, run the following cell to download the training and test data.

In [3]:
!wget https://drive.google.com/uc\?export\=download\&id\=1cJeJjoCfycp4f3yHABO8bai6Em0zoc15 -O train.csv
!wget https://drive.google.com/uc\?export\=download\&id\=1owiCCMlYXdT1z7wdz5k6If1fqQquI76P -O test.csv
!wget https://drive.google.com/uc\?export\=download\&id\=12HEAGnegf8h15M_3osmerN8gthW_ERoJ -O train_seqs.fasta
!wget https://drive.google.com/uc\?export\=download\&id\=1W1LDba5TLJwaNWvMT14QMDOfH6VPL7Wy -O test_seqs.fasta
!wget https://drive.google.com/uc\?export\=download\&id\=1-F1Seb2Fb-QOqBjfkFJIvc479IdTinlT -O ec_numbers.json
!wget https://drive.google.com/uc\?export\=download\&id\=1PlS7kXvcKGNlRa74FapVQ8M4GGcsWDjC -O train_subsample.csv
!wget https://drive.google.com/uc\?export\=download\&id\=1F2WyQV1xBdru3B3QBtDmTSQnxp1ZNSuP -O train_subsample.fasta

--2025-02-25 00:16:49--  https://drive.google.com/uc?export=download&id=1cJeJjoCfycp4f3yHABO8bai6Em0zoc15
Resolving drive.google.com (drive.google.com)... 142.250.141.101, 142.250.141.113, 142.250.141.139, ...
Connecting to drive.google.com (drive.google.com)|142.250.141.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1cJeJjoCfycp4f3yHABO8bai6Em0zoc15&export=download [following]
--2025-02-25 00:16:49--  https://drive.usercontent.google.com/download?id=1cJeJjoCfycp4f3yHABO8bai6Em0zoc15&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.2.132, 2607:f8b0:4023:c0d::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7377785 (7.0M) [application/octet-stream]
Saving to: ‘train.csv’

train.csv           100%[===================>]   7.04M  38.8MB/s   

Import the necessary packages:

In [55]:
# export
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json, os, time
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm
##### DO NOT MODIFY ANYTHING IN THIS CELL #####

Load the training data and take a look at the sequences and EC number labels

In [56]:
df = pd.read_csv('train.csv')
df.head()

,Sequence,EC number,split
0,MTDLGLKWSCEYCTYENWPSAIKCTMCRAQRHNAPIITEEPFKSSS...,3.4.19.12,train
1,MHASLSSWLLAASLLTQPISVSGQGCPFAKRDGTVDSSLPQKRADA...,1.11.1.21,train
2,MSGYSSDRDRGRDRGFGAPRFGGSRAGPLSGKKFGNPGEKLVKKKW...,3.6.4.13,train
3,MTDSGDLCPHLDSIGEVTKEELIQKSKGTCQSCGVGGPNLWACLQC...,3.4.19.12,train
4,MSDEGSKRGSRADSLEAEPPLPPPPPPPPPGESSLVPTSPRYRPPL...,2.3.2.27,train


In [57]:
with open('ec_numbers.json') as f:
    ec_list = json.load(f)
print(f'Number of EC numbers: {len(ec_list)}')

Number of EC numbers: 200


In [58]:
sequences = df['Sequence'].tolist()
ec_numbers = df['EC number'].tolist()
ec2idx = {ec: idx for idx, ec in enumerate(ec_list)}
train_seq2name = {seq: f'train_seq_{i}' for i, seq in enumerate(sequences)}

Split 20% of the training data as the validation set:

In [59]:
seq_train, seq_val, ec_train, ec_val = train_test_split(sequences, ec_numbers, test_size=0.2, random_state=42)
print(f'Training samples: {len(seq_train)}')
print(f'Validation samples: {len(seq_val)}')

Training samples: 16000
Validation samples: 4000


## Task 1: One-hot tokenizer

Protein sequences consist of a list of amino acids. There are 20 types of standard amino acids. We need to transform (tokenize) protein sequences into tensors so that neural networks can take them as inputs. A straightforward way to tokenize protein sequences is to use one-hot encoding ([wiki link](https://en.wikipedia.org/wiki/One-hot)). In this task you need to complete the function `one_hot_encode` which takes a protein sequence (a string of amino acids) of length $L$ and output an one-hot-encoded tensor of shape $L\times 20$ (Note: there exist some unknown amino acids 'X' in the sequences, for such amino acid we can just encode it as all-zero vector).

In [ ]:
# export
##### DO NOT MODIFY ANYTHING ABOVE THIS LINE #####

amino_acids = "ACDEFGHIKLMNPQRSTVWY"
aa_to_idx = {aa: i for i, aa in enumerate(amino_acids)}

# One-hot encoding function
def one_hot_encode(sequence):
    #sequence is a string of amino acids, return the one-hot encoded tensor of the sequence.
    one_hot = torch.zeros((len(sequence), len(amino_acids)))
    for i, aa in enumerate(sequence):
        if aa in aa_to_idx:
            one_hot[i, aa_to_idx[aa]] = 1
    return one_hot

With the one-hot tokenizer, we can design the dataset class. As you can see in the following cell, each data point returned by the dataset class contains three items: the one-hot encoded tensor, the length of the sequence, and the label. Since the length of the sequences in the dataset can vary, we provide the collate function that pads the sequences with zeros to the maximum length in the batch. You can pass this `collate_fn` to the `collate_fn` parameter of pytorch's DataLoader class to ensure the correct behaviour of batching.

In [ ]:
# export
##### DO NOT MODIFY ANYTHING ABOVE THIS LINE #####

class ProteinDataset(Dataset):
    def __init__(self, sequences, labels, ec2idx):
        self.sequences = sequences
        self.labels = [ec2idx.get(ec, -1) for ec in labels]
        #use one_hot_encode to get the sequence tensors
        #####
        self.seq_tensors = [one_hot_encode(seq) for seq in self.sequences]
        #####

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq_tensor = self.seq_tensors[idx]
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return seq_tensor, len(seq_tensor), label

# Collate function with padding
def collate_fn(batch):
    sequences, seq_lens, labels = zip(*batch)
    #max_len = max(seq_lens)

    padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=0)

    return padded_sequences, torch.tensor(seq_lens, dtype=torch.long), torch.tensor(labels, dtype=torch.long)


## Task 2: Transformer built from multi-head attention

In this task, you are required to implement a vanilla transformer encoder model for the EC number prediction task. You should construct the transformer model using blocks like `torch.nn.MultiheadAttention`, `torch.nn.Linear`, `torch.nn.LayerNorm`. Your transformer model should have the same architecture as the encoder module described in the paper [Attention is all you need](https://arxiv.org/abs/1706.03762). We recommend you to check PyTorch's documentation for the modules mentioned before.

In [ ]:
# export
##### DO NOT MODIFY ANYTHING ABOVE THIS LINE #####

class AttentionClassifier(nn.Module):
    def __init__(self, num_classes, embed_dim=128, num_heads=2, num_layers=2, ff_dim=256):
        super(AttentionClassifier, self).__init__()
        self.embedding = nn.Linear(20, embed_dim)
        # implement the transformer block using multiheadattention, linear, and layernorm.
        #####
        self.attention_layers = nn.ModuleList([
            nn.ModuleDict({
                'norm1': nn.LayerNorm(embed_dim),
                'attention': nn.MultiheadAttention(embed_dim, num_heads, batch_first=True),
                'norm2': nn.LayerNorm(embed_dim),
                'ff': nn.Sequential(
                    nn.Linear(embed_dim, ff_dim),
                    nn.GELU(),
                    nn.Linear(ff_dim, embed_dim)
                ),
                'dropout': nn.Dropout(0.1)
            }) for _ in range(num_layers)
        ])
        #####
        self.pooling = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, x, seq_lens):
        x = self.embedding(x)
        max_len = x.shape[1]
        mask = torch.arange(max_len, device=x.device).expand(len(seq_lens), max_len) >= seq_lens.unsqueeze(1)
        mask = mask.to(x.device)

        # forward part of the transformer block
        #####
        for layer in self.attention_layers:
            norm_x = layer['norm1'](x)
            attn_out, _ = layer['attention'](norm_x, norm_x, norm_x, key_padding_mask=mask)
            x = x + layer['dropout'](attn_out)

            norm_x = layer['norm2'](x)
            ff_out = layer['ff'](norm_x)
            x = x + ff_out
        #####

        x = x.permute(0, 2, 1)
        x = self.pooling(x).squeeze(-1)
        return self.fc(x)

## Task 3: Transformer built from TransformerEncoder class

In this task, you will also implement a vanilla transformer model. Instead of constructing the model from small blocks like MultiheadAttention, you should use the wrapped module `torch.nn.TransformerEncoderLayer` and `torch.nn.TransformerEncoder` to directly build the model. We recommend you to check the documentation of these two modules to learn their usage.

In [ ]:
class TransformerClassifier(nn.Module):
    def __init__(self, num_classes, embed_dim=128, num_heads=2, num_layers=2, ff_dim=128):
        super(TransformerClassifier, self).__init__()
        self.embedding = nn.Linear(20, embed_dim)  # Project one-hot input to embedding space
        #transformer block using TransformerEncoderLayer
        #####
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,          # Embedding dimension
            nhead=num_heads,            # Number of attention heads
            dim_feedforward=ff_dim,     # Feedforward network dimension
            batch_first=True,           # Ensures (batch, seq, embed) order
            dropout=0.1                 # Dropout to prevent overfitting
        )

        self.encoder_layers = nn.TransformerEncoder(
            encoder_layer=encoder_layer,
            num_layers=num_layers,
            norm=nn.LayerNorm(embed_dim)  # Final layer normalization
        )
        #####
        self.pooling = nn.AdaptiveAvgPool1d(1)  # Global average pooling
        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, x, seq_lens):
        x = self.embedding(x)  # (batch_size, seq_len, embed_dim)

        # Create attention mask
        max_len = x.shape[1]
        mask = torch.arange(max_len, device=x.device).expand(len(seq_lens), max_len) >= seq_lens.unsqueeze(1)
        mask = mask.to(x.device)

        #forward part of the transformer block
        #####
        x = self.encoder_layers(x, src_key_padding_mask=mask)
        #####

        x = x.permute(0, 2, 1)
        x = self.pooling(x).squeeze(-1)  # (batch, embed_dim)
        return self.fc(x)

## Task 4: 1D-CNN model

In this task, you are going to implement a model using 1D CNN layers. You can use PyTorch's `torch.nn.Conv1d` to construct the model. Note that for simplicity, you do not have to consider the padded part of the input tensor. Refer to PyTorch's documentation for the usage of `torch.nn.Conv1d`.

In [ ]:
class CNNClassifier(nn.Module):
    def __init__(self, num_classes, embed_dim=128, num_filters=128, kernel_size=3, num_layers=4):
        super(CNNClassifier, self).__init__()
        self.embedding = nn.Linear(20, embed_dim)
        #1D convolutional layers
        #####
        self.conv_layers = nn.ModuleList([
            nn.Sequential(
                nn.Conv1d(
                    in_channels=embed_dim if i == 0 else num_filters,  # First layer uses embed_dim, others use num_filters
                    out_channels=num_filters,
                    kernel_size=kernel_size,
                    padding=kernel_size // 2  # Keeps output length same as input
                ),
                nn.BatchNorm1d(num_filters),  # Stabilizes training
                nn.ReLU(),  # Activation function
                nn.Dropout(0.2)  # Prevents overfitting
            ) for i in range(num_layers)
        ])
        #####

        self.pooling = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(num_filters, num_classes)

    def forward(self, x, seq_lens):
        x = self.embedding(x).permute(0, 2, 1)  # Convert to (batch, embed_dim, seq_len)

        #forward part of the CNN block
        #####
        for conv_layer in self.conv_layers:
            x = conv_layer(x)
        #####

        x = self.pooling(x).squeeze(-1)  # Global average pooling
        return self.fc(x)

## Training the neural networks

Complete the function `train_model`.

In [ ]:
def train_model(model, train_dataset, val_dataset, num_classes, epochs=10, batch_size=64, lr=1e-4, patience=10, device='cuda:0',use_mixed_precision=True):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=collate_fn)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    if int(torch.__version__.split('.')[0]) >= 2:  # if torch >= 2.0
        model = torch.compile(model)

    #Mixed precision scaler
    scaler = torch.cuda.amp.GradScaler(enabled=(use_mixed_precision and "cuda:0" in device.type))

    best_acc = 0
    patience_counter = 0
    best_ckpt = None

    for epoch in range(epochs):
        start_epoch = time.time()
        model.train()
        total_loss, correct, total = 0, 0, 0

        for sequences, seq_lens, labels in train_loader:
            #backpropagation
            #####
            sequences = sequences.to(device)
            seq_lens = seq_lens.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            # Forward + backward with AMP
            with torch.cuda.amp.autocast(enabled=(scaler is not None)):
                output = model(sequences, seq_lens)
                loss = criterion(output, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            #####

            total_loss += loss.item()
            preds = output.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = correct / total

        # Validation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for sequences, seq_lens, labels in val_loader:
                # model inference
                #####
                sequences = sequences.to(device)
                seq_lens = seq_lens.to(device)
                labels = labels.to(device)
                with torch.cuda.amp.autocast(enabled=(scaler is not None)):
                    pred = model(sequences, seq_lens)
                preds = pred.argmax(dim=1)
                #####
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        val_acc = correct / total
        end_epoch = time.time()
        print(f'Epoch [{epoch+1} / {epochs}]: Train Loss={total_loss:.4f}, Train Acc={train_acc:.4f}, Val Acc={val_acc:.4f}, Time={end_epoch - start_epoch:.4f} sec')

        # Early stopping
        if val_acc > best_acc:
            best_acc = val_acc
            best_ckpt = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered.")
                break

    return model, best_ckpt

Train AttentionClassifier. Note that to get better performance, you might need to tune the hyperparameters like epochs, batch_size, learning rate (lr), early stop patience (patience), as well as the model size (number of layers, number of attention heads, embed dimension, feed-forward dimensions). The same is true for training other models.

In [ ]:
train_dataset = ProteinDataset(seq_train, ec_train, ec2idx)
val_dataset = ProteinDataset(seq_val, ec_val, ec2idx)
num_classes = len(ec_list)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = AttentionClassifier(num_classes).to(device)
model, best_ckpt = train_model(model, train_dataset, val_dataset, num_classes=num_classes, epochs=30, batch_size=64, lr=1e-3, patience=10, device=device,use_mixed_precision=True)
model.load_state_dict(best_ckpt)

df_test = pd.read_csv('test.csv')
test_sequences = df_test['Sequence'].tolist()
test_seq2name = {seq: f'test_seq_{i}' for i, seq in enumerate(test_sequences)}
test_dataset = ProteinDataset(test_sequences, [0]*len(test_sequences), ec2idx)
test_loader = DataLoader(test_dataset, batch_size=256, collate_fn=collate_fn,num_workers=2)

model.eval()
preds = []
with torch.no_grad():
    for sequences, seq_lens, _ in test_loader:
        #inference on the test set
        #####
        sequences = sequences.to(device)
        seq_lens = seq_lens.to(device)
        with torch.cuda.amp.autocast(enabled=True):
            output = model(sequences, seq_lens)
        batch_preds = output.argmax(dim=1).tolist() # preds should be a list of predicted label indices
        preds.extend(batch_preds)
        #####
# save the predictions to a individual CSV file, each row contains the predicted EC number for the corresponding sequence in the test set, no need for header
preds = [ec_list[pred] for pred in preds]
df_preds = pd.DataFrame(preds)
df_preds.to_csv('test_preds_attention.csv', index=False, header=False)

<ipython-input-12-98a9fa245b6d>:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(use_mixed_precision and "cpu" in device.type))
<ipython-input-12-98a9fa245b6d>:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(scaler is not None)):
<ipython-input-12-98a9fa245b6d>:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(scaler is not None)):
<ipython-input-12-98a9fa245b6d>:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(scaler is not None)):
<ipython-input-12-98a9fa245b6d>:56: FutureWarning: `torch.cuda.amp.autocast(args.

Epoch [1 / 30]: Train Loss=1243.6509, Train Acc=0.0255, Val Acc=0.0845, Time=136.4291 sec


<ipython-input-12-98a9fa245b6d>:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(scaler is not None)):
<ipython-input-12-98a9fa245b6d>:56: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(scaler is not None)):


Epoch [2 / 30]: Train Loss=901.8471, Train Acc=0.1677, Val Acc=0.2707, Time=20.7812 sec
Epoch [3 / 30]: Train Loss=701.5862, Train Acc=0.3159, Val Acc=0.2830, Time=20.9329 sec
Epoch [4 / 30]: Train Loss=629.5336, Train Acc=0.3816, Val Acc=0.4160, Time=20.8448 sec
Epoch [5 / 30]: Train Loss=570.4885, Train Acc=0.4268, Val Acc=0.4535, Time=20.5990 sec
Epoch [6 / 30]: Train Loss=530.4654, Train Acc=0.4679, Val Acc=0.4805, Time=20.8841 sec
Epoch [7 / 30]: Train Loss=483.7121, Train Acc=0.5068, Val Acc=0.5115, Time=20.7374 sec
Epoch [8 / 30]: Train Loss=471.1046, Train Acc=0.5306, Val Acc=0.2447, Time=21.0207 sec
Epoch [9 / 30]: Train Loss=486.6552, Train Acc=0.5051, Val Acc=0.5430, Time=20.1843 sec
Epoch [10 / 30]: Train Loss=406.5377, Train Acc=0.5789, Val Acc=0.5865, Time=20.4832 sec
Epoch [11 / 30]: Train Loss=363.7658, Train Acc=0.6203, Val Acc=0.5905, Time=20.7041 sec
Epoch [12 / 30]: Train Loss=345.4596, Train Acc=0.6327, Val Acc=0.6290, Time=20.7408 sec
Epoch [13 / 30]: Train Loss=3

<ipython-input-13-51c6f418c6a2>:24: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):


Train TransformerClassifier

In [ ]:
model = TransformerClassifier(num_classes).to(device)
model, best_ckpt = train_model(model, train_dataset, val_dataset, num_classes=num_classes, epochs=20, batch_size=64, lr=1e-3, patience=10, device=device,use_mixed_precision=True)
model.load_state_dict(best_ckpt)

model.eval()
preds = []
with torch.no_grad():
    for sequences, seq_lens, _ in test_loader:
        # inference on the test set
        #####
        sequences = sequences.to(device)
        seq_lens = seq_lens.to(device)
        with torch.cuda.amp.autocast(enabled=True):
            output = model(sequences, seq_lens)
        batch_preds = output.argmax(dim=1).tolist() # preds should be a list of predicted label indices
        preds.extend(batch_preds)
        #####
# save the predictions to a individual CSV file, each row contains the predicted EC number for the corresponding sequence in the test set, no need for header
preds = [ec_list[pred] for pred in preds]
df_preds = pd.DataFrame(preds)
df_preds.to_csv('test_preds_transformer.csv', index=False, header=False)

<ipython-input-14-c932d5c27bc1>:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(use_mixed_precision and "cuda:0" in device.type))
<ipython-input-14-c932d5c27bc1>:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(scaler is not None)):
<ipython-input-14-c932d5c27bc1>:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(scaler is not None)):
<ipython-input-14-c932d5c27bc1>:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(scaler is not None)):
<ipython-input-14-c932d5c27bc1>:31: FutureWarning: `torch.cuda.amp.autocast(ar

Epoch [1 / 20]: Train Loss=1256.3697, Train Acc=0.0250, Val Acc=0.0660, Time=109.9625 sec


<ipython-input-14-c932d5c27bc1>:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(scaler is not None)):
<ipython-input-14-c932d5c27bc1>:56: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(scaler is not None)):


Epoch [2 / 20]: Train Loss=911.5446, Train Acc=0.1787, Val Acc=0.2692, Time=12.7497 sec
Epoch [3 / 20]: Train Loss=700.7411, Train Acc=0.3434, Val Acc=0.3937, Time=12.6335 sec
Epoch [4 / 20]: Train Loss=584.3491, Train Acc=0.4366, Val Acc=0.4485, Time=12.5742 sec
Epoch [5 / 20]: Train Loss=494.6092, Train Acc=0.5218, Val Acc=0.5078, Time=12.5075 sec
Epoch [6 / 20]: Train Loss=435.6871, Train Acc=0.5667, Val Acc=0.5327, Time=12.5140 sec
Epoch [7 / 20]: Train Loss=387.0776, Train Acc=0.6115, Val Acc=0.6118, Time=12.4998 sec
Epoch [8 / 20]: Train Loss=351.2817, Train Acc=0.6420, Val Acc=0.6505, Time=12.6649 sec
Epoch [9 / 20]: Train Loss=318.6854, Train Acc=0.6758, Val Acc=0.6530, Time=12.4961 sec
Epoch [10 / 20]: Train Loss=296.1388, Train Acc=0.6985, Val Acc=0.6472, Time=12.6079 sec
Epoch [11 / 20]: Train Loss=279.3431, Train Acc=0.7100, Val Acc=0.6743, Time=12.5414 sec
Epoch [12 / 20]: Train Loss=265.7355, Train Acc=0.7221, Val Acc=0.6937, Time=12.5543 sec
Epoch [13 / 20]: Train Loss=2

<ipython-input-15-32b536fcd393>:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):


Train CNNClassifier

In [ ]:
model = CNNClassifier(num_classes).to(device)
model, best_ckpt = train_model(model, train_dataset, val_dataset, num_classes=num_classes, epochs=30, batch_size=32, lr=1e-3, patience=10, device=device)
model.load_state_dict(best_ckpt)

model.eval()
preds = []
with torch.no_grad():
    for sequences, seq_lens, _ in test_loader:
        # Inference on the test set
        #####
        sequences = sequences.to(device)
        seq_lens = seq_lens.to(device)
        with torch.cuda.amp.autocast(enabled=True):
            output = model(sequences, seq_lens)
        batch_preds = output.argmax(dim=1).tolist() # preds should be a list of predicted label indices
        preds.extend(batch_preds)
        #####
# save the predictions to a individual CSV file, each row contains the predicted EC number for the corresponding sequence in the test set, no need for header
preds = [ec_list[pred] for pred in preds]
df_preds = pd.DataFrame(preds)
df_preds.to_csv('test_preds_cnn.csv', index=False, header=False)

<ipython-input-14-c932d5c27bc1>:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(use_mixed_precision and "cuda:0" in device.type))
<ipython-input-14-c932d5c27bc1>:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(scaler is not None)):
<ipython-input-14-c932d5c27bc1>:56: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(scaler is not None)):


Epoch [1 / 30]: Train Loss=2458.7405, Train Acc=0.0325, Val Acc=0.0465, Time=5.9850 sec
Epoch [2 / 30]: Train Loss=2170.7937, Train Acc=0.0911, Val Acc=0.1153, Time=5.2184 sec
Epoch [3 / 30]: Train Loss=1956.3706, Train Acc=0.1552, Val Acc=0.1185, Time=4.6771 sec
Epoch [4 / 30]: Train Loss=1770.8212, Train Acc=0.2306, Val Acc=0.1298, Time=4.6804 sec
Epoch [5 / 30]: Train Loss=1609.7084, Train Acc=0.3036, Val Acc=0.2510, Time=4.8049 sec
Epoch [6 / 30]: Train Loss=1457.4856, Train Acc=0.3791, Val Acc=0.0628, Time=4.5766 sec
Epoch [7 / 30]: Train Loss=1317.8623, Train Acc=0.4555, Val Acc=0.3195, Time=4.8506 sec
Epoch [8 / 30]: Train Loss=1197.5053, Train Acc=0.5109, Val Acc=0.1348, Time=4.5732 sec
Epoch [9 / 30]: Train Loss=1080.1762, Train Acc=0.5674, Val Acc=0.3380, Time=4.5560 sec
Epoch [10 / 30]: Train Loss=985.2720, Train Acc=0.6105, Val Acc=0.4420, Time=4.7926 sec
Epoch [11 / 30]: Train Loss=893.0555, Train Acc=0.6536, Val Acc=0.1620, Time=4.5232 sec
Epoch [12 / 30]: Train Loss=810.

<ipython-input-39-1c44c8d513df>:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):


## Task 5: Using pretrained protein language model embeddings

In the previous tasks we are using one-hot encoded sequences as the model inputs. With the advancement of language models, many pretrained protein language models (pLM) have been widely used in protein-related problems. Below we are going to explore the usage of pLM embeddings for EC number prediction. We will use ESM-2 (https://github.com/facebookresearch/esm) to extract protein sequence embeddings. First you need to to check ESM-2's documentation to learn how to generate the embeddings using the fasta files we have provided. You should use the model `esm2_t33_650M_UR50D`, retrieve the last-layer sequence-level embedding (no need for residue-level embedding). You should generate one `.pt` file for each sequence embedding and save it in the directory `esm_embeddings`. Since the embedding generation can be time-consuming, we will use a subsampled training set (50% of the original training set) for this task. If you have adquate computational resources, you can also use the complete training set.

In [62]:
!pip install fair-esm
!pip install biopython

In [ ]:
import esm
from Bio import SeqIO
from tqdm.auto import tqdm
import os
import numpy as np
import torch

def gen_emb(fasta_file, out_dir='esm_embeddings', device='cuda:0'):
    records = list(SeqIO.parse(fasta_file, 'fasta'))
    names = [rec.id for rec in records]
    sequences = [str(rec.seq) for rec in records]
    print(f'Number of sequences: {len(sequences)}')

    data = [(name, seq) for name, seq in zip(names, sequences)]

    #Load ESM-2 model (esm2_t33_650M_UR50D) and batch converter
    #####
    model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
    batch_converter = alphabet.get_batch_converter()
    #####
    model.to(device)
    model.eval()  # disables dropout for deterministic results

    batch_size = 16 # Reduce if you are running out of cuda memory
    num_batches = int(np.ceil(len(data) / batch_size))

    for i in tqdm(range(num_batches)):
        batch = data[i * batch_size:(i + 1) * batch_size]
        names_batch, seqs_batch = zip(*batch)
        batch_labels, batch_strs, batch_tokens = batch_converter(batch)
        batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
        batch_tokens = batch_tokens.to(device)
        # Extract per-residue representations (on CPU)
        with torch.no_grad():
            #inference
            #####
            results = model(batch_tokens, repr_layers=[33], return_contacts=True)
            #####
        # get per-residue representations
        #####
        token_representations = results['representations'][33]
        #####
        # Generate per-sequence representations via averaging
        for k, tokens_len in enumerate(batch_lens):
            seq_name = names_batch[k]
            seq_tokens = token_representations[k, :tokens_len]
            seq_mean = seq_tokens.mean(0)
            save = {'mean_representations': {33: seq_mean}}
            path = os.path.join(out_dir, f'{seq_name}.pt')
            print(path)
            torch.save(save, path)

You have two options for getting the ESM-2 embeddings:

Option 1: Generate the embeddings by yourself

You can run the gen_emb function in the following two cells to generate the embeddings.

In [ ]:
gen_emb('test_seqs.fasta')

Number of sequences: 2000


  0%|          | 0/125 [00:00<?, ?it/s]

In [ ]:
gen_emb('train_subsample.fasta')

Number of sequences: 10000


  0%|          | 0/625 [00:00<?, ?it/s]

Option 2: Download the precomputed embeddings

If you have trouble with the GPU or PACE cluster, you can choose to run the following cell to download the precomputed embeddings. Note that you still need to implement the gen_emb function if you use the precomputed embeddings.

In [11]:
#!wget "https://drive.usercontent.google.com/download?id=1wLGtohLE1vdZigOxs9T7o-7STT_Jkpi5&export=download" -O esm_embeddings.zip
!unzip esm_embeddings.zip

Streaming output truncated to the last 5000 lines.
  inflating: esm_embeddings/train_seq_1.pt  
  inflating: esm_embeddings/train_seq_667.pt  
  inflating: esm_embeddings/train_seq_8610.pt  
  inflating: esm_embeddings/train_seq_8111.pt  
  inflating: esm_embeddings/train_seq_18116.pt  
  inflating: esm_embeddings/train_seq_8618.pt  
  inflating: esm_embeddings/train_seq_18586.pt  
  inflating: esm_embeddings/train_seq_11321.pt  
  inflating: esm_embeddings/train_seq_18578.pt  
  inflating: esm_embeddings/train_seq_10227.pt  
  inflating: esm_embeddings/train_seq_12353.pt  
  inflating: esm_embeddings/train_seq_17519.pt  
  inflating: esm_embeddings/train_seq_6949.pt  
  inflating: esm_embeddings/train_seq_10717.pt  
  inflating: esm_embeddings/train_seq_17604.pt  
  inflating: esm_embeddings/test_seq_793.pt  
  inflating: esm_embeddings/train_seq_19151.pt  
  inflating: esm_embeddings/train_seq_8807.pt  
  inflating: esm_embeddings/train_seq_1917.pt  
  inflating: esm_embeddings/train

In [65]:
df = pd.read_csv('train_subsample.csv')
sequences = df['Sequence']
ec_numbers = df['EC number'].tolist()

seq_train, seq_val, ec_train, ec_val = train_test_split(sequences, ec_numbers, test_size=0.2, random_state=42)

Construct a new dataset class to use ESM-2 embeddings.

In [66]:
class ProteinESMDataset(Dataset):
    def __init__(self, sequences, seq2name, emb_dir, labels, ec2idx):
        super().__init__()
        self.labels = [ec2idx.get(ec, -1) for ec in labels]
        self.embeddings = []
        for seq in tqdm(sequences, desc='Loading esm embeddings'):
            name = seq2name[seq]
            emb_file = os.path.join(emb_dir, f'{name}.pt')
            emb = torch.load(emb_file)['mean_representations'][33]
            self.embeddings.append(emb)

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, index):
        emb = self.embeddings[index]
        label = torch.tensor(self.labels[index], dtype=torch.long)
        return emb, label

Implement a simple MLP model to use ESM-2 embeddings as inputs for EC number prediction.

In [ ]:
class MLPClassifier(nn.Module):
    def __init__(self, num_classes, input_dim=1280, hidden_dim=640):
        super(MLPClassifier, self).__init__()
        #linear layers
        self.fc_input = nn.Linear(input_dim, hidden_dim)  # Input layer
        self.batch_norm = nn.BatchNorm1d(hidden_dim)  # Normalization layer
        self.activation_fn = nn.ReLU()  # Activation function
        self.dropout_layer = nn.Dropout(0.1)  # Dropout for regularization
        self.fc_output = nn.Linear(hidden_dim, num_classes)  # Output layer


    def forward(self, x):
        #forward function
        x = self.fc_input(x)
        x = self.batch_norm(x)
        x = self.activation_fn(x)
        x = self.dropout_layer(x)
        x = self.fc_output(x)

        return x

Complete the function `train_model_esm`, which trains the model taking ESM-2 embeddings as inputs.

In [ ]:
def train_model_esm(model, train_dataset, val_dataset, num_classes, epochs=100, batch_size=256, lr=1e-3, patience=10, device='cuda:0', use_mixed_precision = True):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    if int(torch.__version__.split('.')[0]) >= 2:
        model = torch.compile(model)
    scaler = torch.cuda.amp.GradScaler(enabled=(use_mixed_precision and "cuda:0" in device.type))

    best_acc = 0
    patience_counter = 0
    best_ckpt = None

    for epoch in range(epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0

        for sequences, labels in train_loader:
            # backpropogation
            #####
            outputs = None
            loss = None
            sequences = sequences.to(device)
            labels = labels.to(device)
            with torch.cuda.amp.autocast(enabled=(scaler is not None)):
                outputs = model(sequences)
                loss = criterion(outputs, labels)

            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            #####

            total_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = correct / total

        # Validation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for sequences, labels in val_loader:
                # inference
                #####
                preds = None
                sequences = sequences.to(device)
                labels = labels.to(device)
                with torch.cuda.amp.autocast(enabled=True):
                    outputs = model(sequences)
                preds = outputs.argmax(dim=1)
                #####
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        val_acc = correct / total
        print(f'Epoch {epoch+1}: Train Loss={total_loss:.4f}, Train Acc={train_acc:.4f}, Val Acc={val_acc:.4f}')

        # Early stopping
        if val_acc > best_acc:
            best_acc = val_acc
            best_ckpt = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered.")
                break

    return model, best_ckpt

In [ ]:
emb_dir = 'esm_embeddings'
os.makedirs(emb_dir, exist_ok=True)
train_dataset = ProteinESMDataset(seq_train, train_seq2name, emb_dir, ec_train, ec2idx)
val_dataset = ProteinESMDataset(seq_val, train_seq2name, emb_dir, ec_val, ec2idx)
num_classes = len(ec_list)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = MLPClassifier(num_classes).to(device)
model, best_ckpt = train_model_esm(model, train_dataset, val_dataset, num_classes=num_classes, epochs=30, batch_size=32, lr=1e-3, patience=5, device=device)
model.load_state_dict(best_ckpt)

df_test = pd.read_csv('test.csv')
test_sequences = df_test['Sequence'].tolist()
test_seq2name = {seq: f'test_seq_{i}' for i, seq in enumerate(test_sequences)}
test_dataset = ProteinESMDataset(test_sequences, test_seq2name, emb_dir, [0]*len(test_sequences), ec2idx)
test_loader = DataLoader(test_dataset, batch_size=256)

model.eval()
preds = []
with torch.no_grad():
    for sequences, _ in test_loader:
        # inference on the test set
        #####
        sequences = sequences.to(device)
        outputs = model(sequences)
        pred = outputs.argmax(dim=1)
        preds.extend(pred.tolist())
        #####
# save the predictions to a individual CSV file, each row contains the predicted EC number for the corresponding sequence in the test set, no need for header
preds = [ec_list[pred] for pred in preds]
df_preds = pd.DataFrame(preds)
df_preds.to_csv('test_preds_esm.csv', index=False, header=False)

Loading esm embeddings:   0%|          | 0/8000 [00:00<?, ?it/s]

<ipython-input-66-b49496802fb2>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  emb = torch.load(emb_file)['mean_representations'][33]


Loading esm embeddings:   0%|          | 0/2000 [00:00<?, ?it/s]

<ipython-input-71-31f7cf2b8782>:10: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(use_mixed_precision and "cuda:0" in device.type))
<ipython-input-71-31f7cf2b8782>:27: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(scaler is not None)):
<ipython-input-71-31f7cf2b8782>:54: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):


Epoch 1: Train Loss=210.4560, Train Acc=0.8704, Val Acc=0.9835
Epoch 2: Train Loss=18.1885, Train Acc=0.9862, Val Acc=0.9880
Epoch 3: Train Loss=9.5704, Train Acc=0.9935, Val Acc=0.9890
Epoch 4: Train Loss=6.8852, Train Acc=0.9955, Val Acc=0.9880
Epoch 5: Train Loss=9.8298, Train Acc=0.9930, Val Acc=0.9875
Epoch 6: Train Loss=8.1763, Train Acc=0.9946, Val Acc=0.9865
Epoch 7: Train Loss=11.1410, Train Acc=0.9914, Val Acc=0.9840
Epoch 8: Train Loss=3.7754, Train Acc=0.9968, Val Acc=0.9855
Early stopping triggered.


Loading esm embeddings:   0%|          | 0/2000 [00:00<?, ?it/s]

<ipython-input-66-b49496802fb2>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  emb = torch.load(emb_file)['mean_representations'][33]


Train the MLP model and generate predictions for the test set.

## Grading

### Task 1 (4 points)
Task 1 will be graded by the correctness of the function `one_hot_encode`.

### Task 2 (9 points)
Tasks 2-5 will be graded by the accuracy of the predictions made by each model on the test set.

- Accuracy >= 0.7: 9 points
- Accuracy >= 0.65: 8 points
- Accuracy >= 0.6: 7 points
- Accuracy >= 0.55: 6 points
- Accuracy < 0.55: 0 points

### Task 3 (9 points)

- Accuracy >= 0.7: 9 points
- Accuracy >= 0.65: 8 points
- Accuracy >= 0.6: 7 points
- Accuracy >= 0.55: 6 points
- Accuracy < 0.55: 0 points

### Task 4 (9 points)

- Accuracy >= 0.5: 9 points
- Accuracy >= 0.45: 7 points
- Accuracy < 0.45: 0 points

### Task 5 (9 points)

- Accuracy >= 0.97: 9 points
- Accuracy >= 0.96: 7 points
- Accuracy >= 0.95: 5 points
- Accuracy < 0.95: 0 points

## Submission

After completing all the tasks, you need to submit five files to Gradescope:
- `hw3_nn.ipynb`, the notebook file with all tasks completed.
- `test_preds_attention.csv`
- `test_preds_transformer.csv`
- `test_preds_cnn.csv`
- `test_preds_esm.csv`
- `weights.csv`: the answer for Problem 3: Design a Neural Network by Hand.

Note that the four `.csv` files for predictions on the test set will be automatically generated when running this notebook, do not change the codes regarding the save of the prediction results. **DO NOT** submit the files in a zip file, please submit them individually to Gradescope.